In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [10]:
import tensorflow as tf
import numpy as np

In [11]:
tf.logging.set_verbosity(tf.logging.INFO) #DEBUG, INFO, WARN, ERROR, FATAL

In [18]:
IRIS_TRAINING = "iris_training.csv"
IRIS_TEST = "iris_test.csv"

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TRAINING,
                                                                    target_dtype=np.int,
                                                                    features_dtype=np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TEST,
                                                    target_dtype=np.int,
                                                    features_dtype=np.float32)

feature_columns = [tf.contrib.layers.real_valued_column("",dimension=4)]

#控制监控
validation_metrics = {"accuracy":tf.contrib.metrics.streaming_accuracy,
                     "precision":tf.contrib.metrics.streaming_precision,
                     "recall":tf.contrib.metrics.streaming_recall}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=2000)

classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                           hidden_units=[10,20,10],
                                           n_classes=3,
                                           model_dir="iris_model",
                                           config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))
classifier.fit(x=training_set.data,
              y=training_set.target,
              steps=2000,
              monitors=[validation_monitor])

accuracy_score = classifier.evaluate(x=test_set.data, y=test_set.target)["accuracy"]

print('Accuracy:{0:f}'.format(accuracy_score))

new_samples = np.array([[6.4,3.2,4.5,1.5],[5.8,3.1,5.0,1.7]],dtype=float)
y = list(classifier.predict(new_samples, as_iterable=True))
print('Predictions:{}'.format(str(y)))


INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, 'save_checkpoints_secs': 1, 'tf_config': intra_op_parallelism_threads: 4
inter_op_parallelism_threads: 4
gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, 'tf_random_seed': 42, 'num_ps_replicas': 0}
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Restored model from iris_model/model.ckpt-8000-?????-of-00001
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32)
INFO:tensorflow:Restored model from iris_model/model.ckpt-8000-?????-of-00001
INFO:tensorflow:Eval steps [0,inf) for training step 8000.
INFO:tensorflow:Input iterator is exhausted.
INFO:tensorflow:Saving evaluation summary for 8000 step: recall = 1.0, accuracy = 0.966667

Accuracy:0.966667
Predictions:[1, 1]
